In [ ]:
import geemap
import ee
Map = geemap.Map()
Map

In [ ]:
roi = ee.Geometry.Rectangle([93, 37, 103, 40], None, False)
Map.addLayer(roi, {}, "roi")

In [ ]:
def cloudMask(image):
    qa = image.select('QA_PIXEL')
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 3)\
          .And(qa.bitwiseAnd(1 << 8))\
          .Or(qa.bitwiseAnd(1 << 4))
    # 删除不出现在所有波段的边缘像元
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

def NDSI1(image):
    img = image.expression(
        '((green - sho) / (green + sho))', {
      'green': image.select("SR_B2"),
      'sho': image.select("SR_B5")
    }).rename('NDSI')
    # img.updateMask(img.gt(0.5).And(img.lt(0.8)))
    # img.updateMask(img.gt(0.5))
    return image.addBands(img)

def NDSI2(image):
    img = image.expression(
        '((green - sho) / (green + sho))', {
      'green': image.select("SR_B3"),
      'sho': image.select("SR_B6")
    }).rename('NDSI')
    # img.updateMask(img.gt(0.5).And(img.lt(0.8)))
    # img.updateMask(img.gt(0.5))
    return image.addBands(img)

In [ ]:
for year in range(1986,2022):
    start = str(year)
    end = str(year+1)
    collection4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")\
                    .filterBounds(roi)\
                    .filterDate(start,end)\
                    .map(cloudMask).map(NDSI1)
    collection5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
                    .filterBounds(roi)\
                    .filterDate(start,end)\
                    .map(cloudMask).map(NDSI1)
    collection7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")\
                    .filterBounds(roi)\
                    .filterDate(start,end)\
                    .map(cloudMask).map(NDSI1)
    collection8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                    .filterBounds(roi)\
                    .filterDate(start,end)\
                    .map(cloudMask).map(NDSI2)
    col1 = ee.ImageCollection(collection4.merge(collection5))
    col2 = ee.ImageCollection(col1.merge(collection7))
    collection = ee.ImageCollection(col2.merge(collection8))
    img = collection.select('NDSI').mean()
    img1 = img.updateMask(img.gt(0.5).And(img.lt(0.8)))
    img1 = img1.multiply(1000).toInt().divide(1000)
    outfile = r"E:\冰川/"+start
    #     clip_dow_merge(roi, img1, outfile, 30)
    from tolocal import export
    export(region=roi, image=img1,
    outfile=outfile, scale=30, sep=0.1)